In [6]:
import numpy as np
from llime_explainer import LLiMeExplainer

In [7]:
N = 10 # Parameter n, number of neighbors per class to generate
P = 1.0 # Percentage of Relevant Word w to forward to the LLM for both classes
R = 5 # Parameter r, max number of query

In [8]:
model_name = "deepseek/deepseek-v3.2"
api_key = ""
one_neigh_prompt_path = "prompts/OneWordNeighborGenerationSystemPrompt.txt"
neighs_prompt_path = "prompts/NeigborsGenerationSystemPrompt.txt"
system_prompt_path = "prompts/SystemCounterfactualGenerationPrompt.txt"

sentence = "I love to watch football history channel, it's my favorite sport"
label = 0
labels = ["Sports", "Music & Entertainment"]

def dummy_classifier(sentences):
    to_ret = []
    for sentence in sentences:
        if 'football' in sentence or 'sport' in sentence:
            to_ret.append([0.9,0.1])
        else: to_ret.append([0.25,0.75])
    return np.array(to_ret)

explainer = LLiMeExplainer(model_name, api_key, one_neigh_prompt_path, neighs_prompt_path, system_prompt_path)

In [9]:
local_surrogate, vocabulary, x, y, neighbors = explainer.get_local_surrogate(sentence, labels, dummy_classifier, N, R)

In [10]:
explanation = explainer.get_explaination(local_surrogate, vocabulary, x, label, 0.5)
print(explanation)

{'tokens': ['football', 'me', 'channel', 'hobby', 'musical', 'entertainment', 'show', 'is', 'history', 'in', 'music', 'interest', 'the', 'follow', 'watch', 'genre', 'broadcast', 'program', 'listen', 'sport'], 'coeff': [-0.8839531044867948, 0.36796221606898805, -0.30084064382829584, 0.14767696245154188, 0.16657644100947516, 0.5234521586683796, 0.14503124724885835, -0.09549546048185167, -0.20103639261575826, 0.1274057305185363, 0.932829705893444, 0.16034884804683547, 0.627764929906732, 0.16034884804683547, -0.4789629396465961, 0.17628463577840806, 0.14767696245154188, 0.15162136750767316, 0.17628463577840806, -0.965099811017651]}


In [11]:
explainer.get_counterfactual(sentence, explanation, label, labels, dummy_classifier, R)

("I love to watch music history channel, it's my favorite show",
   OPERATION OUT_WORD   IN_WORD
 0       SUB    music  football
 1       REM              sport
 2       INS               show)